In [27]:
''' 
negocios_csv.ipynb
# captura de un archivo desde un contenedor en Azure Storage como formato csv y cargarlo en SQL Server
Autor: Patricio Araneda
Fecha: 2023-09-25
'''
import os
import io
from pathlib import Path
from urllib.parse import urlparse
import pyodbc
import sqlalchemy as sal
from sqlalchemy import text
from sqlalchemy import create_engine
from azure.storage.blob import BlobClient
from azure.storage.blob import BlobServiceClient
import pandas as pd
from pd_to_mssql import to_sql

connect_str = os.environ["AZURE_STORAGE_CONNECTION_STRING"]


In [31]:
#
def azure_download_csv_to_df(url=None):
    if url:
        blob_service_client = BlobServiceClient.from_connection_string(connect_str)
        path = urlparse(url).path
        path = path.split("/")
        container = path[1]
        blob = '/'.join(path[2:])
        blob_client = blob_service_client.get_blob_client(container=container, blob=blob)
        with io.BytesIO() as input_blob:
            blob_client.download_blob().download_to_stream(input_blob)
            input_blob.seek(0)
            df = pd.read_csv(input_blob, sep=";")
        return df
    else:
        return None

In [ ]:
# df from BLOB
df = azure_download_csv_to_df("url en azure blob storage.csv")
df.drop(index=0, inplace=True)
df= pd.DataFrame(df)
df.head()

In [ ]:
blob = BlobClient.from_connection_string(conn_str=connect_str, container_name="cubit", blob_name="filename.csv")

with open("./negocios.csv", "wb") as my_blob:
    blob_data = blob.download_blob()
    blob_data.readinto(my_blob)
data = pd.read_csv("./negocios.csv", sep=";")
data.drop(index=0, inplace=True)
df.head()

In [ ]:
# acceso a FTP
data = pd.read_csv("file.csv", sep=";")
data.drop(index=0, inplace=True)
#data.to_csv('negocios.csv', index=False, sep=";", encoding="utf-8")
data.describe()

In [ ]:
#carga de datos en tabla DW
server = os.environ["sql_host"]
database = os.environ["sql_db"] 
username = os.environ["sql_user"]
password = os.environ["sql_pwd"]

conn_str = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';AUTOCOMMIT=FALSE;UID='+username+';PWD='+ password

to_sql(data, 'tablename', conn_str, schema='dbo', index=True, replace=True, chunk_size=1000, thread_count=5, ignore_truncation=False, ignore_missing=False)
# ODBC Driver 17 for SQL Server